# multi-layer perception on mnist

## load package

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../MNIST_data/', one_hot=True)
%matplotlib inline  
print ("PACKAGES LOADED")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
PACKAGES LOADED


## define model

In [2]:
# network topologies
n_hidden_1 = 256
n_hidden_2 = 128
n_input = 784
n_classes = 10

# input and outputs
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# network parameters
stddev = 0.1
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])), 
    'b2': tf.Variable(tf.random_normal([n_hidden_2])), 
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print("network ready")

Instructions for updating:
Colocations handled automatically by placer.
network ready


## MLP as a function

In [3]:
def multilayer_perception(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    return (tf.matmul(layer_2, _weights['out']) + _biases['out'])

## define function

In [5]:
# # prediction and loss
# pred = tf.nn.softmax(multilayer_perception(x, weights, biases))
# loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))

pred = multilayer_perception(x, weights, biases)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

# optimizer
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# accuracy
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))  # 返回bool向量
accuracy = tf.reduce_mean(tf.cast(corr, "float"))  # bool向量转float, 求平均值

# init
init = tf.global_variables_initializer()
print("function ready")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

function ready


## run

In [6]:
# parameter
training_epochs = 20
batch_size      = 100
display_step    = 4

# session graph
sess = tf.Session()
sess.run(init)

# train
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optimizer, feed_dict=feeds)
        avg_cost += sess.run(loss, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    
    # display
    if (epoch+1) % display_step == 0:        
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accuracy, feed_dict=feeds)        
        test_feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accuracy, feed_dict=test_feeds)
        print ("Epoch: %03d/%03d cost: %.9f  " "train accuracy: %.3f  " "test accuracy: %.3f"
               % (epoch + 1, training_epochs, avg_cost, train_acc, test_acc))   
        
print ("optimize done")

Epoch: 004/020 cost: 0.120837435  train accuracy: 0.990  test accuracy: 0.963
Epoch: 008/020 cost: 0.046590915  train accuracy: 1.000  test accuracy: 0.977
Epoch: 012/020 cost: 0.019108779  train accuracy: 1.000  test accuracy: 0.978
Epoch: 016/020 cost: 0.006669461  train accuracy: 1.000  test accuracy: 0.978
Epoch: 020/020 cost: 0.003286693  train accuracy: 1.000  test accuracy: 0.980
optimize done


In [7]:
# close 
sess.close()